# Generate Pydantic Output

In [1]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [7]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

model_chat = ChatOpenAI()

In [22]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


class Jokes(BaseModel):
    Jokes: List[Joke]

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a 3 jokes."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Jokes)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

output = model_chat(_input.to_messages())

res = parser.parse(output.content)

In [26]:
JK = res.Jokes

In [31]:
for i in JK:
    print(i.punchline)

Because they make up everything!
Fsh!
They don't have the guts!


In [21]:
_input.to_messages()

[HumanMessage(content='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke.\n', additional_kwargs={}, example=False)]

In [6]:
parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')